In [2]:
import os
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk-11.0.13'
os.environ['SPARK_HOME'] = 'C:\spark\spark-3.4.1-bin-hadoop3'

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [4]:
path = r'C:\repos\portfolio_projetos\block_fraud_company\data\raw'
file = r'\PS_20174392719_1491204439457_log.csv'

In [5]:
spark = SparkSession.builder.getOrCreate()

In [6]:
df = spark.read.csv(path + file, header = True, inferSchema=True)

In [7]:
df.printSchema()

root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- isFlaggedFraud: integer (nullable = true)



In [8]:
df.count()

6362620

In [9]:
df.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|      

In [10]:
df1 = df.groupBy('type').count().withColumnRenamed('count','somatoria')
df1.withColumn('percentage', 100*(f.col('somatoria') / df1.select(f.sum('somatoria')).collect()[0][0])).show()

+--------+---------+------------------+
|    type|somatoria|        percentage|
+--------+---------+------------------+
|TRANSFER|   532909| 8.375621992198184|
| CASH_IN|  1399284|21.992261049693365|
|CASH_OUT|  2237500| 35.16633085112736|
| PAYMENT|  2151495| 33.81460781879163|
|   DEBIT|    41432|0.6511782881894566|
+--------+---------+------------------+



In [11]:
df1 = df.groupBy('isFraud').count().withColumnRenamed('count','somatoria')
# df1.show()
df1.withColumn('percentage',100*(f.col('somatoria') / df1.select(f.sum('somatoria')).collect()[0][0])).show()

+-------+---------+-------------------+
|isFraud|somatoria|         percentage|
+-------+---------+-------------------+
|      1|     8213|0.12908204481801522|
|      0|  6354407|  99.87091795518198|
+-------+---------+-------------------+



In [21]:
df.select('amount',(f.round(f.log(f.col('amount') ),3).alias('log_amount'))).corr('amount','amount')

1.0

In [22]:
df.select('amount',f.round(f.log(f.col('amount')),3).alias('log_amount')).cov('amount','log_amount')

424363.5357953158

In [27]:
df.orderBy(f.asc('type'),f.desc('isFraud')).show()

+----+-------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|   type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+-------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
| 395|CASH_IN| 32363.46|C1951289658|      19213.0|      51576.46| C823288423|     154596.75|     122233.29|      0|             0|
| 202|CASH_IN|273937.49|C1004815319|   1987422.41|     2261359.9| C453399078|     608723.54|     334786.04|      0|             0|
|  40|CASH_IN|330186.02|C1446132520|     198459.0|     528645.02| C794873386|     6726601.4|    6396415.38|      0|             0|
| 202|CASH_IN|207540.39|C1757042042|   5267463.18|    5475003.57| C436595683|    2165620.42|    1958080.03|      0|             0|
| 156|CASH_IN| 27287.59|C1634109320|   8673566.52|    8700854.11|C1542157370|     5

In [36]:
df.select('type','isFraud').where('isFraud = 1').show()

+--------+-------+
|    type|isFraud|
+--------+-------+
|TRANSFER|      1|
|CASH_OUT|      1|
|TRANSFER|      1|
|CASH_OUT|      1|
|TRANSFER|      1|
|CASH_OUT|      1|
|CASH_OUT|      1|
|TRANSFER|      1|
|CASH_OUT|      1|
|TRANSFER|      1|
|CASH_OUT|      1|
|TRANSFER|      1|
|CASH_OUT|      1|
|CASH_OUT|      1|
|TRANSFER|      1|
|CASH_OUT|      1|
|TRANSFER|      1|
|CASH_OUT|      1|
|TRANSFER|      1|
|CASH_OUT|      1|
+--------+-------+
only showing top 20 rows



In [37]:
df.select('type','isFraud').where(f.col('isFraud') == 0).show()

+--------+-------+
|    type|isFraud|
+--------+-------+
| PAYMENT|      0|
| PAYMENT|      0|
| PAYMENT|      0|
| PAYMENT|      0|
| PAYMENT|      0|
| PAYMENT|      0|
| PAYMENT|      0|
|   DEBIT|      0|
|   DEBIT|      0|
| PAYMENT|      0|
| PAYMENT|      0|
| PAYMENT|      0|
| PAYMENT|      0|
|CASH_OUT|      0|
| PAYMENT|      0|
| PAYMENT|      0|
| PAYMENT|      0|
|TRANSFER|      0|
| PAYMENT|      0|
|   DEBIT|      0|
+--------+-------+
only showing top 20 rows



In [41]:
df.select('type',f.when(f.col('type') == 'PAYMENT','OK').otherwise('NOK').alias('teste')).show()

+--------+-----+
|    type|teste|
+--------+-----+
| PAYMENT|   OK|
| PAYMENT|   OK|
|TRANSFER|  NOK|
|CASH_OUT|  NOK|
| PAYMENT|   OK|
| PAYMENT|   OK|
| PAYMENT|   OK|
| PAYMENT|   OK|
| PAYMENT|   OK|
|   DEBIT|  NOK|
|   DEBIT|  NOK|
| PAYMENT|   OK|
| PAYMENT|   OK|
| PAYMENT|   OK|
| PAYMENT|   OK|
|CASH_OUT|  NOK|
| PAYMENT|   OK|
| PAYMENT|   OK|
| PAYMENT|   OK|
|TRANSFER|  NOK|
+--------+-----+
only showing top 20 rows

